# Encoder multi-head attention

Ya hemos vísto cómo funciona el `Scaled Dot-Product Attention`, por lo que ya podemos abordar el `Multi-Head Attention` del encoder

<div style="text-align:center;">
  <img src="Imagenes/transformer_architecture_model_encoder_multi_head_attention.png" alt="Multi-Head Attention" style="width:425px;height:626px;">
  <img src="Imagenes/multi-head_attention.png" alt="Multi-Head Attention" style="width:501px;height:623px;">
</div>

Viendo la arquitenctura, podemos ver que `K`, `Q` y `V` pasan por un bloque `Linear` y además vemo que tanto los bloques `Linear` y el bloque `Scaled Dot-Product Attention` se repite varias veces, vamos a ver por qué se hace esto

En primer lugar hay que explicar qué es el bloque `Linear` que se ve en la arquitectura, pues es simplemente una capa fully connected de las que explicamos al principio del todo, y ahora veremos por qué

## Proyecciones de las entradas

<div style="text-align:center;">
  <img src="Imagenes/multi-head_attention_proyection.png" alt="Proyection Multi-Head Attention" style="width:501px;height:623px;">
</div>

Como hemos explicado, en la capa de `Input Embedding` se convierte el token a un vector de `n` dimensiones. De manera que por ejemplo puede que una dimensión determine si una palabra es un objeto inanimado o un ser vivo, otra podría capturar si es un objeto tangible o un concepto abstracto, otra podría capturar si la palabra tiene connotaciones positivas o negativas, mientras que otra podría capturar si una palabra es un sustantivo, verbo, adjetivo, etc., otra dimensión podría capturar si la palabra está en singular o plural y otra dimensión podría capturar el tiempo verbal (presente, pasado, futuro). Las tres primeras dimensiones capturan información de la semántica de la palabra, mientras que las tres últimas capturan información de la sintaxis

Por lo que a la hora de introducir las matrices `K`, `Q` y `V` al `Scaled Dot-Product Attention` a lo mejor es mejor que entren juntas las tres primeras dimensiones y por otro lado las tres últimas

Sin embargo, no sabemos exactamente cómo el input embedding construye las dimensiones. En nuestro caso de momento estamos cogiendo el `input embedding` ya entrenado de BERT, por lo que podríamos hacer un estudio, pero son 768 dimensiones, por lo que sería mucho trabajo hacer el estudio de cada dimensión y luego buscar relaciones entre las dimensiones. Pero es que además, en la realidad, el `input embedding` no está entrenado, va cambiando sus pesos durante el entrenamiento, por lo que ni podríamos hacer ese estudio.

Por lo que aquí entran las capas `Linear` que se ven en la arquitectura, nosotros decidimos en cuantos grupos vamos a dividir el embedding, esto lo determina la `h` que se ve a la derecha del `Scaled Dot-Product Attention`. Y esas capas `Linear`, simplemente serán matrices que se quedarán con unas dimensiones u otras, incluso con parte de unas o de otras, puede ser una mezcla. Pero lo importante, es que nosotros no decidimos qué dimeniosnes van con otras, sino que se hace automáticamente durante el entrenamiento del Transformer, de manera que se obtenga el mejor resultado posible

## Concatenación

<div style="text-align:center;">
  <img src="Imagenes/multi-head_attention_concat.png" alt="Concat Multi-Head Attention" style="width:501px;height:623px;">
</div>

Si en las proyecciones se divide el embedding en pequeñas dimensiones, ahora habrá que juntar toda esa información nuevamente, por eso se hace una concatenación de todas las mtrices resultantes del `Scaled Dot-Product Attencion` en la capa de `Concat`

## Linear final

<div style="text-align:center;">
  <img src="Imagenes/multi-head_attention_linear.png" alt="Linear Multi-Head Attention" style="width:501px;height:623px;">
</div>

Por último se pasa todo por un bloque `Linear`, es decir, por una red fully connected. Esto es porque al concatenar las salidas en el paso anterior se obtiene una matriz mucho más grande. Supongamos una frase con 12 tokens y en la que se han obtenido los embeddings con el `input embedding` de BERT, que tiene una dimensión de 768. Por lo que deberíamos tener una matriz de 12x768. Si hacemos 6 proyecciones, es decir, si ponemos a la `h` que sale a la derecha del `Scaled Dot-Product Attention` un valor de 6, vamos a tener 6 matrices de 12x768. Si las concatenamos vamos a tener una matriz de 72x768, pero a la salida del `Multi-Head Attention` deberíamos tener una matriz de 12x768. Por lo que pasamos esta matriz por una red fully connected para a la salida volver a tener una matriz de 12x768

## Implementación

Vamos a implementar la clase `Multi-Head Attention` con Pytorch

Primero volvemos a poner el código de la clase `Scaled Dot-Product Attention` que hemos hecho en el notebook anterior

In [1]:
import torch
import torch.nn as nn
import math

class ScaledDotProductAttention(nn.Module):
    def __init__(self, dim_embedding):
        """
        Args:
            dim_embedding: dimension of embedding vector
        """
        super().__init__()
        self.dim_embedding = dim_embedding
    
    def forward(self, key, query, value):
        """
        Args:
            key: key vector
            query: query vector
            value: value vector
        
        Returns:
            output vector from scaled dot product attention
        """
        # MatMul
        key_trasposed = key.transpose(-1,-2)
        product = torch.matmul(query, key_trasposed)
        # scale
        product = product / math.sqrt(self.dim_embedding)
        # softmax
        attention_matrix = torch.nn.functional.softmax(product, dim=-1)
        # MatMul
        output = torch.matmul(attention_matrix, value)
        
        return output

Y ahora creamos la clase del `Multi-Head Attention`

In [26]:
class MultiHeadAttention(nn.Module):
    def __init__(self, heads, dim_embedding):
        super().__init__()
        
        self.dim_embedding = dim_embedding
        self.dim_proyection = dim_embedding // heads
        self.heads = heads
        
        self.proyection_Q = nn.Linear(dim_embedding, dim_embedding)
        self.proyection_K = nn.Linear(dim_embedding, dim_embedding)
        self.proyection_V = nn.Linear(dim_embedding, dim_embedding)
        self.attention = nn.Linear(dim_embedding, dim_embedding)

        self.scaled_dot_product_attention = ScaledDotProductAttention(self.dim_proyection)
    
    def forward(self, Q, K, V):
        batch_size = Q.size(0)
        
        # perform linear operation and split into h heads
        proyection_Q = self.proyection_Q(Q).view(batch_size, -1, self.heads, self.dim_proyection)
        proyection_K = self.proyection_K(K).view(batch_size, -1, self.heads, self.dim_proyection)
        proyection_V = self.proyection_V(V).view(batch_size, -1, self.heads, self.dim_proyection)
        
        # transpose to get dimensions bs * h * sl * d_model
        proyection_Q = proyection_Q.transpose(1,2)
        proyection_K = proyection_K.transpose(1,2)
        proyection_V = proyection_V.transpose(1,2)

        # calculate attention using function we will define next
        outputs = []
        for i in range(self.heads):
            output = self.scaled_dot_product_attention(proyection_Q[:, i, :, :], proyection_K[:, i, :, :], proyection_V[:, i, :, :])
            outputs.append(output)
        scaled_dot_product_attention = torch.stack(outputs, dim=2)  # stacking along the head dimension
        
        # concatenate heads and put through final linear layer
        concat = scaled_dot_product_attention.transpose(1,2).contiguous().view(batch_size, -1, self.dim_embedding)
        
        output = self.attention(concat)
    
        return output

Vamos a obtener el embedding de una frase

In [27]:
import torch
from transformers import BertModel, BertTokenizer

def extract_embeddings(input_sentence, model_name='bert-base-uncased'):
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)
    
    input_ids = tokenizer.encode(input_sentence, add_special_tokens=True)
    input_ids_tensor = torch.tensor([input_ids])
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    
    with torch.no_grad():
        outputs = model(input_ids_tensor)
        
    token_embeddings = outputs[0][0]
    
    # Los embeddings posicionales están en la segunda capa de los embeddings de la arquitectura BERT
    positional_encodings = model.embeddings.position_embeddings.weight[:len(input_ids), :].detach()
    
    embeddings_with_positional_encoding = token_embeddings + positional_encodings
    
    return tokens, input_ids, token_embeddings, positional_encodings, embeddings_with_positional_encoding

In [28]:
sentence1 = "I gave the dog a bone because it was hungry"
tokens1, input_ids1, token_embeddings1, positional_encodings1, embeddings_with_positional_encoding1 = extract_embeddings(sentence1)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Instanciamos un objeto de la clase `Multi-Head Attention`

In [29]:
dim_embedding = embeddings_with_positional_encoding1.shape[-1]
heads = 8
multi_head_attention = MultiHeadAttention(heads=heads, dim_embedding=dim_embedding)

In [30]:
attention = multi_head_attention(embeddings_with_positional_encoding1, embeddings_with_positional_encoding1, embeddings_with_positional_encoding1)
attention.shape

torch.Size([12, 1, 768])

A la salida obtenemos una matriz de 12x1x768 en vez de 12x768 por la línea `concat = scaled_dot_product_attention.transpose(1,2).contiguous().view(batch_size, -1, self.dim_embedding)`, para obtener 12x768 debería ser `concat = scaled_dot_product_attention.transpose(1,2).contiguous().view(batch_size, self.dim_embedding)`, es decir, quitar el `-1`. Sin embargo lo hemos dejado así porque luego nos será útil para el entrenamiento, para poder meter un dataloader con varias sentencias